In [1]:
import openai
import io
import pandas as pd
import tqdm

## OpenAI Vector Store

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client = OpenAI()


In [ ]:
vector_store = client.vector_stores.create(
    name="hot_links"
)
print(vector_store.id)

# Create the vector store

vs_68acb4af008c81919b4924472e209446


In [ ]:
def make_file_object(name,content):
    return (name+'.txt', io.BytesIO(content.encode('utf-8')), 'text/markdown')
# Important to encode as bytes and add '.txt' to filename to trick API that this is a file

In [ ]:
def push_file_to_cloud(name, content):
    result = client.files.create(file = make_file_object(name, content), purpose = 'assistants') 
    return result

In [ ]:
def add_file_to_db(file_id, attributes = {}):
    result = client.vector_stores.files.create(
        vector_store_id=vector_store.id,
        file_id=file_id,
        attributes=attributes
    )
    return result
# Then add it to the vector store with attributes

VectorStoreFile(id='file-RJNaMAB5ipLTiv696Tai1e', created_at=1756149658, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_68acb4af008c81919b4924472e209446', attributes={'a2': 'test'}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))


In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    input="What is wine made from?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "filters": {
            "type": "eq",
            "key": "a2",
            "value": "test"
        }
    }]
)
print(response)
# Then query the store with filters

Response(id='resp_68acb79ea0a481968e1f2be3956ea55107af2b659c522373', created_at=1756149662.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_68acb79ff0d48196a30ae2e48f26285b07af2b659c522373', queries=['What is wine made from?'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_68acb7a14c348196bcc4a429678b855307af2b659c522373', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-RJNaMAB5ipLTiv696Tai1e', filename='dummy_4.txt.txt', index=344, type='file_citation')], text='According to the search results in your file, wine is made from plums. However, traditionally, wine is most commonly made from fermented grapes, although it can be made from various fruits, including plums, apples, and others. In the snippet, there is a suggestion that wine is made from plums, perhaps humorously or as an example of fruit wine. \

In [ ]:
def get_all_vs():
    for f in client.vector_stores.list():
        yield f
# Get all vector stores

VectorStore(id='vs_68acb4af008c81919b4924472e209446', created_at=1756148911, file_counts=FileCounts(cancelled=0, completed=5, failed=0, in_progress=0, total=5), last_active_at=1756149657, metadata={}, name='hot_links', object='vector_store', status='completed', usage_bytes=5246, expires_after=None, expires_at=None, description=None)
VectorStore(id='vs_68acae441e188191a605f82b03a98305', created_at=1756147268, file_counts=FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3), last_active_at=1756148425, metadata={}, name='hot_links', object='vector_store', status='completed', usage_bytes=3131, expires_after=None, expires_at=None, description=None)
VectorStore(id='vs_68ac553757d88191873c6900f9c90c0c', created_at=1756124471, file_counts=FileCounts(cancelled=0, completed=994, failed=6, in_progress=0, total=1000), last_active_at=1756146900, metadata={}, name='hot_links_final2', object='vector_store', status='completed', usage_bytes=6545010, expires_after=None, expires_at=None

In [12]:
for f in tqdm.tqdm(client.files.list()):
    #print(f.id)

    res = client.files.delete(f.id)
    
    if not res.deleted:
        tqdm.tqdm.write(f"Failed to delete file {f.id}")

1217it [10:56,  1.85it/s]


In [60]:
for i in tqdm.tqdm(range(100)):
    result = client.files.create(file = make_file_object('dummy_{:d}.txt'.format(i), "Wine is made from plums... right?"), purpose='assistants')


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:52<00:00,  1.90it/s]


Based on [OpenAI documentation](https://platform.openai.com/docs/guides/tools-file-search#how-to-use)

In [14]:
b = io.BytesIO(b"some initial text data")

In [16]:
b.getvalue()

b'some initial text data'

In [22]:
from openai.types import FileObject

In [24]:
?FileObject

Init signature:
FileObject(
    *,
    id: str,
    bytes: int,
    created_at: int,
    filename: str,
    object: Literal['file'],
    purpose: Literal['assistants', 'assistants_output', 'batch', 'batch_output', 'fine-tune', 'fine-tune-results', 'vision', 'user_data'],
    status: Literal['uploaded', 'processed', 'error'],
    expires_at: Optional[int] = None,
    status_details: Optional[str] = None,
    **extra_data: Any,
) -> None
Docstring:     
!!! abstract "Usage Documentation"
    [Models](../concepts/models.md)

A base class for creating Pydantic models.

Attributes:
    __class_vars__: The names of the class variables defined on the model.
    __private_attributes__: Metadata about the private attributes of the model.
    __signature__: The synthesized `__init__` [`Signature`][inspect.Signature] of the model.

    __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.
    __pydantic_core_schema__: The core schema of the model.
   

In [45]:
io.BytesIO('test'.encode('utf-8'))

In [14]:
make_file_object('dummy_3.txt', "Wine is made from grapes")

('dummy_3.txt.txt', <_io.BytesIO at 0x76e349562ca0>, 'text/markdown')

In [28]:
result.id

'file-RJNaMAB5ipLTiv696Tai1e'

In [6]:
def add_file_object(file_object):
    client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=file_object   
)

In [7]:
client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=make_file_object('dummy_3.txt', "Wine is made from grapes")   
)

VectorStoreFile(id='file-Vqdvu9xspCKE49JZ6WoYFr', created_at=1756148927, last_error=None, object='vector_store.file', status='completed', usage_bytes=1048, vector_store_id='vs_68acb4af008c81919b4924472e209446', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [8]:
client.files.retrieve("file-Vqdvu9xspCKE49JZ6WoYFr")

FileObject(id='file-Vqdvu9xspCKE49JZ6WoYFr', bytes=24, created_at=1756148925, filename='dummy_3.txt.txt', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)

In [9]:
results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="How is wine made?",
)

In [10]:
results

SyncPage[VectorStoreSearchResponse](data=[VectorStoreSearchResponse(attributes={}, content=[Content(text='Wine is made from grapes', type='text')], file_id='file-Vqdvu9xspCKE49JZ6WoYFr', filename='dummy_3.txt.txt', score=0.5208766983986627)], object='vector_store.search_results.page', search_query=['How is wine made?'], has_more=False, next_page=None)

In [65]:
??make_file_object

Signature: make_file_object(name, content)
Docstring: <no docstring>
Source:   
def make_file_object(name,content):
    return (name, io.BytesIO(content.encode('utf-8')), 'text/plain')
File:      /tmp/ipykernel_709/3135118408.py
Type:      function

In [11]:
add_file_object(make_file_object('xxx','lorem ipsum'))

In [ ]:
add_file_object(make_file_object('xxx',df.head()['bodyContent'].values[0]))

BadRequestError: Error code: 400 - {'error': {'message': 'File type not supported', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [85]:
df.head(10).apply(lambda row: make_file_object(row['webUrl'], row['bodyContent']), axis=1).apply(add_file_object)